# SBERT Siamese Semantic search model for document search

In [1]:
import os
import re
import glob
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from tqdm import tqdm
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [2]:
def absoluteFilePaths(directory):
    path = []
    files = []
    for dirpath, dirname, filenames in os.walk(directory):
        for f in filenames:
            if not os.path.basename(dirpath).startswith('.'):
                path.append(dirpath)
                files.append(f)
            
    return path, files

In [3]:
# question = "What is attention mechanism"
# true_answer = "The attention mechanism is a part of a neural architecture that enables to dynamically highlight relevant features of the input data, which, in NLP, is typically a sequence of textual elements. It can be applied directly to the raw input or to its higher level representation."

# question = "What is quantum entanglement"
# true_answer = "Quantum Entanglement allows qubits that are separated by incredible distances to interact with each other instantaneously (not limited to the speed of light)."

# question = "What are the applications of Face Swapping"
# true_answer = "Face swapping has a number of compelling applications in video compositing, transfiguration in portraits, and especially in  identity  protection  as  it  can  replace  faces  in  photographs by ones from a collection of stock images"

question = "Find everything on face swapping"

In [4]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [5]:
DIRECTORY = os.getcwd()
locations, documents = absoluteFilePaths(os.path.join(DIRECTORY, 'Google', 'research'))
paths = [os.path.join(loc, doc) for loc, doc in zip(locations, documents)]

In [6]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [7]:
contents = []
for path in paths:
    if path.endswith('.pdf'):
        contents.append(convert_pdf_to_txt(path))
    else: 
        continue

## SBERT  

In [8]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [9]:
content_dict = {}

for path, content in zip(paths, contents):
    i=0
    for sentence in content.split('.'):
        content_dict[path+'_'+str(i)] = [sentence, embedder.encode(sentence)]
        i += 1

In [10]:
question_embedding = embedder.encode(question)

distances = {}
for k, val in content_dict.items():
    distances[k] = scipy.spatial.distance.cdist([question_embedding], [list(val)[1]], "cosine")[0]

In [11]:
import operator
newA = dict(sorted(distances.items(), key=operator.itemgetter(1), reverse=True)[:5])

In [12]:
newA

{'/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf_246': array([1.04054031]),
 '/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf_42': array([1.02204708]),
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf_432': array([0.99941252]),
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf_511': array([0.99498056]),
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf_191': array([0.99288753])}

In [13]:
content_dict[list(newA.keys())[1]][0]

', creating a fake bridge across a river although there is no\nsuch a bridge in reality'